# Feature Engineering

In this notebook, we will create and refine features to improve model performance. 

In [1]:
import pandas as pd

# I. Data daily

In [2]:
# Đọc dữ liệu train test

In [3]:
# Tính toán các feature

In [4]:
# Lưu dữ liệu tập train test để training

# II. Data hourly

In [5]:
# Đọc dữ liệu train test

In [6]:
# Tính toán các feature

In [7]:
# Lưu dữ liệu tập train test để training